In [1]:
import os
import fitsio
import numpy as np
from astropy.table import Table, vstack
from astropy.io import fits
from desispec.io import read_spectra
from desitrip.preproc import rebin_flux, rescale_flux

In [2]:
# tiles from that prog with observation *only* that night, i.e. designed that night
def get_tileids_fafns(night, prog):
    assert prog in ["bright", "dark"]
    # tiles from that prog with observation *only* that night, i.e. designed that night
    d = Table.read("/global/cfs/cdirs/desi/spectro/redux/daily/exposures-daily.csv")
    black_expids = np.unique(d["EXPID"][d["NIGHT"] != night])
    sel = (d["NIGHT"] == night) & (d["EFFTIME_SPEC"] > 0)
    sel &= d["FAFLAVOR"] == "main{}".format(prog)
    sel &= ~np.in1d(d["EXPID"], black_expids)
    d = d[sel]
    # unique list of tiles
    _, ii = np.unique(d["TILEID"], return_index=True)
    d = d[ii]
    # fiberassign file paths (looking in spectro/data, as they may not be in svn yet)
    fafns = []
    for i in range(len(d)):
        expdir = os.path.join(
            os.getenv("DESI_ROOT"),
            "spectro",
            "data",
            str(night),
            "{:08d}".format(d["EXPID"][i]),
        )
        fafn = os.path.join(expdir, "fiberassign-{:06d}.fits.gz".format(d["TILEID"][i]))
        assert os.path.isfile(fafn)
        fafns.append(fafn)
    fafns = np.array(fafns)
    #
    return d["TILEID"], fafns

In [3]:
def report_night(night, prog, t):

    assert prog in ["bright", "dark"]

    d = Table()
    d["TILEID"], d["FAFN"] = get_tileids_fafns(night, prog)

    # now check fiberassign against ToOs
    d["ASSGN_NTOO"], d["POT_NTOO"] = 0, 0
    d["ASSGN_TIMESTAMP_MIN"] = np.zeros_like(t["TIMESTAMP"], shape=(len(d),))
    d["ASSGN_TIMESTAMP_MAX"] = np.zeros_like(t["TIMESTAMP"], shape=(len(d),))
    assgn_tids, pot_tids = [], []

    for i in range(len(d)):

        # read fiberassign
        assgn_d = fitsio.read(d["FAFN"][i], "FIBERASSIGN")
        pot_d = fitsio.read(d["FAFN"][i], "TARGETS")

        # potentia / assigned ToOs
        sel = np.in1d(t["TARGETID"], assgn_d["TARGETID"])
        d["ASSGN_NTOO"][i] = sel.sum()
        assgn_tids += t["TARGETID"][sel].tolist()
        if sel.sum() > 0:
            d["ASSGN_TIMESTAMP_MIN"][i] = sorted(t["TIMESTAMP"][sel])[0]
            d["ASSGN_TIMESTAMP_MAX"][i] = sorted(t["TIMESTAMP"][sel])[-1]
        sel = np.in1d(t["TARGETID"], pot_d["TARGETID"])
        d["POT_NTOO"][i] = sel.sum()
        pot_tids += t["TARGETID"][sel].tolist()

    # tiles from one night should not overlap!
    assert np.unique(assgn_tids).size == len(assgn_tids)
    assert np.unique(pot_tids).size == len(pot_tids)

    # report
    r = Table()
    r["NIGHT"], r["PROG"], r["NTILE"] = [night], [prog.upper()], [len(d)]
    r["POT_NTOO"], r["ASSGN_NTOO"] = len(pot_tids), len(assgn_tids)
    ts_mins = d["ASSGN_TIMESTAMP_MIN"][d["ASSGN_TIMESTAMP_MIN"] != ""]
    if ts_mins.size == 0:
        r["ASSGN_TIMESTAMP_MIN"] = "-"
    else:
        r["ASSGN_TIMESTAMP_MIN"] = sorted(ts_mins)[0]
    ts_maxs = d["ASSGN_TIMESTAMP_MIN"][d["ASSGN_TIMESTAMP_MIN"] != ""]
    if ts_maxs.size == 0:
        r["ASSGN_TIMESTAMP_MAX"] = "-"
    else:
        r["ASSGN_TIMESTAMP_MAX"] = sorted(ts_maxs)[-1]

    return r, d

In [4]:
progs = ["bright"]

# ToOs (starting in Dec. 2024)
fn = os.path.join(
    os.getenv("DESI_SURVEYOPS"), "mtl", "main", "ToO", "ToO-fiber.ecsv"
)
t = Table.read(fn)
yms = np.array([int(_[:7].replace("-", "")) for _ in t["TIMESTAMP"]])
sel = yms >= 202412
t = t[sel]
print(
    "found BRIGHT={} and DARK={} ToOs".format(
        (t["OCLAYER"] == "BRIGHT").sum(), (t["OCLAYER"] == "DARK").sum()
    )
)

# nights (starting in Dec. 2024)
d = Table.read("/global/cfs/cdirs/desi/spectro/redux/daily/exposures-daily.csv")
sel = d["EFFTIME_SPEC"] > 0
sel &= (d["FAFLAVOR"] == "mainbright") | (d["FAFLAVOR"] == "maindark")
d = d[sel]
nights = np.unique(d["NIGHT"])
nights = nights[nights >= 20241201]

for prog in progs:
    sel = t["OCLAYER"] == prog.upper()
    rs = []
    for night in nights:
        r, d = report_night(night, prog, t[sel])
        rs.append(r)
    r = vstack(rs)
    print("")
    r.pprint_all()
    print("")

found BRIGHT=1415 and DARK=0 ToOs

 NIGHT    PROG  NTILE POT_NTOO ASSGN_NTOO    ASSGN_TIMESTAMP_MIN       ASSGN_TIMESTAMP_MAX   
-------- ------ ----- -------- ---------- ------------------------- -------------------------
20241201 BRIGHT     5        0          0                         -                         -
20241202 BRIGHT     5        0          0                         -                         -
20241203 BRIGHT     4        1          1 2024-12-03T15:48:31+00:00 2024-12-03T15:48:31+00:00
20241204 BRIGHT     6        0          0                         -                         -
20241205 BRIGHT     6        8          2 2024-12-03T15:48:31+00:00 2024-12-03T15:48:31+00:00
20241206 BRIGHT     8        3          0                         -                         -
20241207 BRIGHT     4        2          1 2024-12-06T18:54:27+00:00 2024-12-06T18:54:27+00:00
20241208 BRIGHT     8        1          0                         -                         -
20241209 BRIGHT     9    

In [5]:
r, d = report_night(20241207, prog, t[sel])

In [6]:
d[3]

TILEID,FAFN,ASSGN_NTOO,POT_NTOO,ASSGN_TIMESTAMP_MIN,ASSGN_TIMESTAMP_MAX
,,,,s,s
int64,str80,int64,int64,str25,str25
27307,/global/cfs/cdirs/desi/spectro/data/20241207/00267232/fiberassign-027307.fits.gz,1,1,2024-12-06T18:54:27+00:00,2024-12-06T18:54:27+00:00


In [19]:
spectra = fits.open("/global/cfs/cdirs/desi/spectro/data/20241207/00267232/desi-00267232.fits.fz", mode='readonly')

In [29]:
spectra

[<astropy.io.fits.hdu.image.PrimaryHDU object at 0x7f4f87c22830>, <astropy.io.fits.hdu.compressed.compressed.CompImageHDU object at 0x7f4f8c2175b0>, <astropy.io.fits.hdu.compressed.compressed.CompImageHDU object at 0x7f4f87ca8f40>, <astropy.io.fits.hdu.compressed.compressed.CompImageHDU object at 0x7f4f86197610>, <astropy.io.fits.hdu.compressed.compressed.CompImageHDU object at 0x7f4f87ca99f0>, <astropy.io.fits.hdu.compressed.compressed.CompImageHDU object at 0x7f4f87ca8760>, <astropy.io.fits.hdu.compressed.compressed.CompImageHDU object at 0x7f4f87ca8ca0>, <astropy.io.fits.hdu.compressed.compressed.CompImageHDU object at 0x7f4f87ca9c60>, <astropy.io.fits.hdu.compressed.compressed.CompImageHDU object at 0x7f4f87ca9540>, <astropy.io.fits.hdu.compressed.compressed.CompImageHDU object at 0x7f4f861e83a0>, <astropy.io.fits.hdu.compressed.compressed.CompImageHDU object at 0x7f4f861e85b0>, <astropy.io.fits.hdu.compressed.compressed.CompImageHDU object at 0x7f4f861e8a00>, <astropy.io.fits.hdu.

In [17]:
Table(spectra[5].data)

TARGETID,FIBER,LOCATION
int64,int32,int32
39628435957485436,400,0
39628435957485736,400,0
39628435957485772,400,0
39628435957485814,400,0
39628435957485825,400,0
39628435957486030,400,0
39628435957486035,400,0
39628435957486109,400,0
616089188260905766,400,0
